In [3]:
# inference.py
import pickle
import numpy as np

import json

def model_fn(model):
    with open(f"{model}/cosine_sim.pkl", "rb") as f:
        cosine_sim = pickle.load(f)
    with open(f"{model}/indices.pkl", "rb") as f:
        indices = pickle.load(f)
    with open(f"{model}/tfidfvectorization.pkl", "rb") as f:
        tfidf = pickle.load(f)
    with open(f"{model}/movies.pkl", "rb") as f:
        movies = pickle.load(f)
    return (cosine_sim, indices, tfidf, movies)

def input_fn(request_body, request_content_type):
    if request_content_type == 'application/json':
        input_data = json.loads(request_body)
        return input_data['title'], input_data.get('n', 5)
    raise ValueError("Unsupported content type: {}".format(request_content_type))

def predict_fn(input_data, model):
    title, n = input_data
    cosine_sim, indices, tfidf, movies = model
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n+1]  # skip first as it's the movie itself
    movie_indices = [i[0] for i in sim_scores]
    recommendations = movies['title'].iloc[movie_indices].tolist()
    return recommendations

def output_fn(prediction, content_type):
    return json.dumps({"recommendations": prediction}), content_type
